# **NN** Implementation (Forward + Backward Propagation)
1. Add input in /test/input and model in /test/model with .txt format, makesure they both have the same filename.
2. Input the filename for both the input and the model, ensuring that they have the same filename.
3. Scroll down and click on `./tmp/network.html`. This will redirect you to network.html, which you can then open the visualization using a live server.

#### Made by:
- Samuel Christoper Swandi - 13520075
- Grace Claudia - 13520078
- Ubaidillah Ariq Prathama - 13520085
- Patrick Amadeus Irawan - 13520109

------------

List of Content

1. [Library & Dependencies](#library)
2. [Helper Function](#helper)
3. [Neural Network Visualization](#visualization)

## Library & Dependencies <a name="library"></a>

In [ ]:
!python3 -m pip install pyvis==0.3.2
!python3 -m pip install networkx==2.6.3
!python3 -m pip install numpy==1.21.6

In [2]:
import numpy as np
from pandas import DataFrame
from sklearn.neural_network import MLPClassifier
import os

from pyvis.network import Network

## **Class**

#### Connected Layer

In [3]:
class ConnectedLayer:
    def __init__(self, input_size, output_size, weights):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = weights[1:]
        self.bias = weights[0]

    def forward(self, input):
        self.input = input
        return np.dot(input, self.weights) + self.bias

#### Activation Layer

In [4]:
class ActivationLayer:
    def __init__(self, activation):
        self.activation = activation
    
    def forward(self, input):
        self.input = input
        return self.activation(input)

#### BackPropagation Supporting Function

Compute Output Gradient (dE_dW)

In [186]:
def grad_output(sample_y_pred, sample_y_train, sample_pred_out, act_type):
    if act_type == "linear": #linear
        dO_dNet = 1
    elif act_type == "ReLU": #relu
        dO_dNet = np.where(sample_y_pred <= 0, 0, 1)
    elif act_type == "sigmoid": #sigmoid
        dO_dNet = sample_y_pred * (1 - sample_y_pred)

    dE_dO = -(sample_y_train - sample_y_pred)
    dNet_dW = sample_pred_out                    
    dE_dW = dE_dO * dO_dNet * dNet_dW

    print(f"\n--Calculation {act_type}---")
    print("dE_dO")
    print(dE_dO)
    print("dO_dNet")
    print(dO_dNet)
    print("dNet_dW")
    print(dNet_dW)
    print("dE_dW")
    print(dE_dW)

    return dE_dW, dE_dO * dO_dNet
    

Compute Hidden Gradient (dEtotal_dW)

In [160]:
def grad_hidden(succ_dE_dNet, succ_weight, succ_out, sample_pred_out, act_type):
    if act_type == "linear": #linear
        dH_dNet = 1
    elif act_type == "ReLU": #relu TODO: keknya sala harusnya y, bukan act(y)
        dH_dNet = np.where(succ_out < 0, 0, 1)
    elif act_type == "sigmoid": #sigmoid
        dH_dNet = (succ_out * (1 - succ_out)).flatten()
    
    # TODO: SOFTMAX

    dE_dNet = succ_dE_dNet
    dNet_dH = succ_weight
    dEtotal_dH = np.sum(dE_dNet * dNet_dH, axis = 1)
    dNet_dW = sample_pred_out
    dEtotal_dNet = dEtotal_dH * dH_dNet
    dEtotal_dW = dEtotal_dNet * dNet_dW

    # print("dEtotal_dH")
    # print(dEtotal_dH)
    # print("dH_dNet")
    # print(dH_dNet)
    # print("dNet_dW")
    # print(dNet_dW)
    # print("dEtotal_dNet")
    # print(dEtotal_dNet)
    # print("dEtotal_dW")
    # print(dEtotal_dW)

    return dEtotal_dW, dEtotal_dNet

#### Neural Network

In [232]:
from math import ceil 

class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)
    
    def predict(self, x):
        complete_result = []
        output = x[:, 1:]

        for layer in self.layers:
            output = layer.forward(output)
            if isinstance(layer, ActivationLayer):
                complete_result.append(output)

        return complete_result[-1] , complete_result
    
    def print_weight(self):
        i = 0
        for layer in self.layers:
            if not isinstance(layer, ActivationLayer):
                print("|____Layer {}___|".format(i))
                print("---B I A S---")
                print(layer.bias)
                print("---N E U R O N S---")
                print(layer.weights)
                i+=1
            print()

    def fit(self, x_train, y_train, learning_rate=0.1, epochs = 5, batch_size = 2, err_threshold = 0.01):
        # batch size validation
        if batch_size > len(x_train) or batch_size <= 0:
            print("Batch size {} invalid.".format(batch_size))
            print("Batch size range [{}...{}]".format(1,len(x_train)))
            return

        print("WEIGHT ORI")
        self.print_weight()
        print("initial error=", sse(predict_output(x_train, self)[0], y_train))
    
        # if initial error already below threshold, stop training
        if sse(predict_output(x_train, self)[0], y_train) <= err_threshold:
            print("Initial error already below threshold, no further training needed.")
            return

        # print layer type for ActivationLayer only
        act_layers = []
        for i in range(len(self.layers)):
            if isinstance(self.layers[i], ActivationLayer):
                act_layers.append(self.layers[i].activation.__name__)
        act_layers.reverse()
        print("\nActivation Layers From Back to Front: ", act_layers)

        print()
        # training loop
        for i in range(epochs):  # Epoch Loop
            # print("- - - - - - - - - - - - - - - - EPOCH {} - - - - - - - - - - - - - - - -".format(i+1))
            # err = 0
            start_i = 0
            end_i = batch_size

            for j in range(ceil(len(x_train) / batch_size)):  # Mini Batch Loop
                print("============BATCH {}==============".format(j + 1))
                batch_x_train = x_train[start_i:end_i]
                batch_y_train = y_train[start_i:end_i]


                # FORWARD PROPAGATION
                batch_y_pred, complete_out = predict_output(batch_x_train, self)
                batch_y_pred = np.array(batch_y_pred)
                # err += sse(batch_y_pred, batch_y_train)

                print("====FORWARD PROPAGATION====")
                print("Batch X Train")
                print(batch_x_train)
                print("Batch Y Train")
                print(batch_y_train)
                print("Batch Y pred")
                print(batch_y_pred)

                # BACKWARD PROPAGATION
                out_grad = None
                first_hid_grad = True
                hid_grad = []

                for k in range(len(batch_x_train)):  # Single Sample Backpropagation Loop
                    sample_y_pred = batch_y_pred[k]
                    sample_y_train = batch_y_train[k]

                    #-------------- Output Gradient --------------#
                    if len(self.layers) == 2:
                        prev_out = batch_x_train[k][1:].reshape(-1,1)
                    else:
                        prev_out = uniform2D(np.array(complete_out)[k, -2]).reshape(-1,1)
                    
                    # Duplicate predecessor to match output layer neuron size & append bias
                    succ_size = len(complete_out[0][-1])
                    sample_pred_out = np.hstack([prev_out] * succ_size)
                    sample_pred_out = np.vstack([np.ones(succ_size),sample_pred_out])

                    print("\n===Batch {} Sample {}===".format(j+1,k+1))

                    print("\n--Output Materials--")
                    print("sample_y_pred")
                    print(sample_y_pred)
                    print("sample_y_train")
                    print(sample_y_train)
                    print("sample_pred_out")
                    print(sample_pred_out)
                    print("Output Layer Type")
                    print(act_layers[0])

                    dE_dW, succ_dE_dNet = grad_output(sample_y_pred, sample_y_train, sample_pred_out, act_layers[0]) # TODO CHANGE NOT SIGMOID
                    if out_grad is None:
                        out_grad = dE_dW
                    else:
                        out_grad += dE_dW

                    # -------------- Hidden Gradient --------------#
                    hid_grad_i = 0
                    reversed_i = -2

                    # TODO : Testing
                    for l_i in range(len(list_layer_size) - 1):
                        if l_i == len(list_layer_size) - 2:
                            prev_out = np.array(batch_x_train[k][1:]).reshape(-1,1)  # cut the bias first
                        else:
                            prev_out = uniform2D(np.array(complete_out)[k, reversed_i]).reshape(-1,1)
                        
                        succ_out = np.array(complete_out)[k, reversed_i].reshape(-1,1)
                        succ_weight = np.array(self.layers[reversed_i].weights)
                        # Duplicate predecessor to match succ layer neuron size & append bias
                        succ_size = len(complete_out[0][reversed_i])
                        sample_pred_out = np.hstack([prev_out] * succ_size)
                        sample_pred_out = np.vstack([np.ones(succ_size),sample_pred_out])

                        # print("\n--Hidden Materials--")
                        # print("sample_pred_out")
                        # print(sample_pred_out)
                        # print("succ_out")
                        # print(succ_out)
                        # print("succ_dE_dNet")
                        # print(succ_dE_dNet)
                        # print("succ_weight")
                        # print(succ_weight)

                        dEtotal_dW, succ_dEtotal_dNet = grad_hidden(succ_dE_dNet, succ_weight, succ_out, sample_pred_out, act_layers[l_i + 1])
                        if first_hid_grad:
                            hid_grad.append(dEtotal_dW)
                        else:
                            hid_grad[hid_grad_i] += dEtotal_dW
                            hid_grad_i += 1

                    first_hid_grad = False
                
                # print("Out grad")
                # print(out_grad)

                out_delta = -learning_rate * out_grad
                hid_delta = -learning_rate * np.array(hid_grad)

                print()
                print("Out_delta")
                print(out_delta)
                print("Hid_delta")
                print(hid_delta)

                # reverse hidden delta
                deltas = (hid_delta[::-1].tolist() + [out_delta.tolist()])
                for d_i in range(len(deltas)):
                    deltas[d_i] = np.array(deltas[d_i])

                # Update weight
                d_i = 0
                for layer in self.layers:
                    if isinstance(layer, ActivationLayer):
                        print(layer.activation.__name__)
                    if not isinstance(layer, ActivationLayer):
                        layer.bias += deltas[d_i][0]
                        print("Layer Weights")
                        print(layer.weights)
                        print("Delta")
                        print(deltas[d_i][1:])
                        layer.weights += deltas[d_i][1:]
                        d_i+=1  

                # Increment next batch
                start_i += batch_size
                end_i += batch_size
                # Verbose Weight per mini-batch
                # self.print_weight()


            # calculate average error on all samples
            # print("X_train")
            # print(x_train)
            # print("Pred")
            # print(predict_output(x_train, self)[0])
            # print("Ori")
            # print(y_train)
            err = sse(predict_output(x_train, self)[0], y_train)

            print("Epoch {}/{}   error={}".format(i+1,epochs, err))
            if err < err_threshold:
                print("Curr error < threshold. Finishing Epoch")
                break
        print("\n----------------------------------------------------------")
        print("============|==========FINAL WEIGHT===========|===========")
        print("----------------------------------------------------------")

        self.print_weight()



In [8]:
def uniform2D(arr):
    return np.array([np.array(i) for i in arr])

## **Helper** Function <a class="anchor" id="helper"></a>

#### Activation Function

In [9]:
def linear(net):
    return net

def ReLU(net):
    return np.maximum(0,net)

def sigmoid(net):
    return 1/(1+np.exp(-net))

def softmax(net):
    res = []
    for sample in net:
        res.append(np.exp(sample)/np.sum(np.exp(sample)))
    return np.array(res)

#### Loss Function

In [144]:
def mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

def sse(y_true, y_pred):
    return np.sum(np.square(y_true - y_pred)) / 2

#### Model Loading Function

In [11]:
def model_load(data):
    '''
    Function to load model from file
    INPUT :     data -> data from model file

    OUTPUT :    weights -> weights of each model neuron
                list_prev_size -> list of previous layer size
                list_layer_size -> list of current layer size
                list_activation -> list of activation function
    '''

    idx = 1
    weights,list_prev_size,list_layer_size,list_activation = [],[],[],[]

    for i in range(int(data[0]) - 1):
        # Loading size & act function
        prev_size, layer_size, activation = [int(i) for i in data[idx].split()]
        list_prev_size.append(prev_size)
        list_layer_size.append(layer_size)
        list_activation.append(activation)
        
        # Loading weights
        idx += 1
        weight = []
        for j in range(prev_size + 1):
            weight.append([float(i) for i in data[idx].split()])
            idx += 1
        weights.append(weight)
    
    # Verbose Information
    print('---Model Information---')
    print('Number of layers :', len(list_prev_size) + 1)
    print('Input size :', list_prev_size[0])
    print('Output size :', list_layer_size[-1])

    print()
    print('Weights :', weights)
    print('Previous layer size :', list_prev_size)
    print('Current layer size :', list_layer_size)
    print('Activation function :', list_activation)

    return weights, list_prev_size, list_layer_size, list_activation

#### Input Loading Function

In [12]:
def generate_x_train(input_data):
    x_train = []
    # Parse each line
    for i in range(len(input_data)):
        x_train.append([float(i) for i in input_data[i].split()])

    return np.array(x_train)

#### Predict Output Function

In [13]:
def predict_output(x, net):
    '''
    Function to predict output from input data
    INPUT :     x-> input data
                net -> NeuralNetwork object

    OUTPUT :    out -> output of the model
                complete_out -> complete output of the model, visualization purpose
    '''
    # Predict output
    out, complete_out = net.predict(x)

    # Gather complete output
    n_complete_out = []
    for i in range(len(complete_out[0])):
        n_complete_out.append([complete_out[0][i], complete_out[-1][i]])

    complete_out = n_complete_out
    return out, complete_out

#### SSE Errors Function + microhelper

In [14]:
def compute_out_and_errors(output_data, out):
    '''
    Function to calculate output error
    INPUT :     output_data -> output data from output file
                out -> output of the model, obtained from predict_output function

    OUTPUT :    out_pred -> output of the model (flattened)
                out_true -> output from output_data (flattened)
                sse_error -> sum squared error
                sse_error <= max_sse -> boolean value, True if sse_error <= max_sse, False otherwise
    '''
    # Assign y_train from output_data
    parsed_output = [[float(j) for j in i.split()] for i in output_data]
    out_pred = out.flatten()
    out_true = np.array(parsed_output[:-1]).flatten()
    max_sse = parsed_output[-1][0]

    return out_pred, out_true, sse(out_true, out_pred), sse(out_true, out_pred) <= max_sse

## **Load** Section

Entrypoint

In [12]:
MODEL_FOLDER = '../test/model/'
INPUT_FOLDER = '../test/input/'
OUTPUT_FOLDER = '../test/output/'
# read file from test folder
def read_file(folder_path, file_name):
    with open(folder_path + file_name, 'r') as file:
        data = [i.rstrip("\n") for i in file.readlines()]
    return data

filename = input('Enter test case name (with extension): ')
data = read_file(MODEL_FOLDER, filename)
input_data = read_file(INPUT_FOLDER, filename)

try: 
    output_data = read_file(OUTPUT_FOLDER, filename)
except:
    output_data = None

model_load

In [13]:
weights, list_prev_size, list_layer_size, list_activation = model_load(data)

---Model Information---
Number of layers : 3
Input size : 2
Output size : 2

Weights : [[[0.35, 0.35], [0.15, 0.25], [0.2, 0.3]], [[0.6, 0.6], [0.4, 0.5], [0.45, 0.55]]]
Previous layer size : [2, 2]
Current layer size : [2, 2]
Activation function : [2, 2]


Build NN

In [14]:
# Build NN model
act = {0: linear, 1: ReLU, 2: sigmoid, 3: softmax}
net = NeuralNetwork()
for i in range (int(data[0]) - 1):
    net.add(ConnectedLayer(list_prev_size[i], list_layer_size[i], weights[i]))
    net.add(ActivationLayer(act[list_activation[i]]))

## **Predict** Section

Prepare x_train & predict output

In [15]:
x_train = generate_x_train(input_data)
out, complete_out = predict_output(x_train, net)

# Print the information, complete with brief verbose
print('---Prediction Information---')
print('Input data :', x_train)
print('Output data :', out)

---Prediction Information---
Input data : [[1.   0.05 0.1 ]]
Output data : [[0.75136507 0.77292847]]


## Compute Error with **Sum Squared Error (SSE)**

Sum Squared Error (SSE) is a mathematical function used in statistics and machine learning to measure the difference between predicted and actual values. It is commonly used as a cost function in various optimization algorithms, such as gradient descent.

The SSE is calculated by taking the difference between each predicted value and its corresponding actual value, squaring the difference, and then summing all of the squared differences:

$$SSE = \sum_{i=1}^{n}(y_i - \hat{y}_i)^2$$

Where:
- $n$ is the number of data points
- $y_i$ is the actual value of the i-th data point
- $\hat{y}_i$ is the predicted value of the i-th data point

The SSE gives an indication of how well the model fits the data. A lower SSE indicates that the model is a better fit for the data.

In machine learning, the SSE is often used as a cost function to be minimized during training of a model. The goal is to find the set of model parameters that minimizes the SSE, thus improving the accuracy of the model's predictions.

In [16]:
out_pred, out_true, sse_err, isLessThanMaxSSE = compute_out_and_errors(output_data, out)

print('Output prediction :', out_pred)
print('Output true :', out_true)
print('SSE  : ', sse_err)
print("sse <= max_sse  :", isLessThanMaxSSE)

Output prediction : [0.75136507 0.77292847]
Output true : [0.01 0.99]
SSE  :  0.5967422175200054
sse <= max_sse  : False


## Neural Network Visualization with **Pyvis**

Pyvis is a Python library that provides an easy-to-use interface for visualizing complex networks, including neural networks. Implementation of the visualization is enlisted below:

In [16]:
def visualize_network(data, complete_out) -> Network:
    n = int(data[0])

    # Constant
    XSTEP, YSTEP, SIZE = 300, 300, 10

    # Nodes
    nodes = []
    node_i = 1

    # Nodes Value
    value = []
    x_val = 0
    y_val = 0

    # Position
    x = []
    y = []

    # Styling + Text
    label = []
    color = []
    edge = []
    title = []

    # Indexing
    src_idx = 0
    idx = 1

    for i_layer in range(n - 1):
        n_curr, n_next, _ = [int(i) for i in data[idx].split()]

        if i_layer == 0:  # means that this is the first layer, hence construct input
            for i_node in range(n_curr + 1):
                if i_node == 0:  # bias
                    color.append("#dd4b39")
                    label.append("Input[bias]")
                    temp = ""
                    for i in range (len(input_data)):
                        temp += str(x_train[i][0])
                        if i < len(input_data) - 1:
                            temp += ", "
                    title.append(temp)
                else:
                    color.append("#162347")
                    label.append("Input[{}]".format(i_node))
                    temp = ""
                    for i in range (len(input_data)):
                        temp += str(x_train[i][i_node])
                        if i < len(input_data) - 1:
                            temp += ", "
                    title.append(temp)
                value.append(SIZE)
                x.append(x_val)
                y.append(y_val)
                y_val += YSTEP
                nodes.append(node_i)
                node_i += 1
            x_val += XSTEP

        y_val = 0
        # always construct the next layer
        for i_node in range(n_next + 1):
            if i_node == 0:
                if i_layer == n - 2:
                    continue
                color.append("#dd4b39")
                label.append("HL{}[bias]".format(i_layer + 1))
                temp = ""
                for i in range (len(input_data)):
                    temp +=  "1"
                    if i < len(input_data) - 1:
                        temp += ", "
                title.append(temp)
            else:
                color.append("#162347")
                if i_layer == n - 2:
                    label.append("Output[{}]".format(i_node))
                else:
                    label.append("HL{}[{}]".format(i_layer + 1, i_node))
                temp = ""
                for i in range (len(input_data)):
                    temp += str(complete_out[i][i_layer][i_node - 1])
                    if i < len(input_data) - 1:
                        temp += ", "
                title.append(temp)
            value.append(SIZE)
            x.append(x_val)
            y.append(y_val)
            y_val += YSTEP
            nodes.append(node_i)
            node_i += 1
        x_val += XSTEP

        idx += 1
        for origin in range(n_curr + 1):
            dst_idx = -1
            for w in reversed(data[idx].split()):
                edge.append((nodes[src_idx], nodes[dst_idx], w))
                dst_idx -= 1
            src_idx += 1
            idx += 1

    g = Network(notebook=True, cdn_resources="remote")
    g.add_nodes(nodes,title = title,value = value,x=x,y=y,label = label,color = color)

    for e in edge:
        g.add_edge(e[0], e[1], title = e[2], color="#162347")

    for n in g.nodes:
        n.update({'physics': False})
    
    return g

Visualize using Helper Function

In [17]:
g = visualize_network(data, complete_out)
g.show("./tmp/network.html")

./tmp/network.html


## **Backpropagation** Section

In [227]:
MODEL_FOLDER = '../test/model/'
INPUT_FOLDER = '../test/input/'
OUTPUT_FOLDER = '../test/output/'
# read file from test folder
def read_file(folder_path, file_name):
    with open(folder_path + file_name, 'r') as file:
        data = [i.rstrip("\n") for i in file.readlines()]
    return data

TC_B = ""
tc_b = input("Directly Enter if you wish to access Test Case B folder (otherwise type anything): ")
if tc_b == "":
    MODEL_FOLDER += "B/"
    INPUT_FOLDER += "B/"
    OUTPUT_FOLDER += "B/"

filename = input('Enter test case name (with extension): ')
data = read_file(MODEL_FOLDER, filename)
input_data = read_file(INPUT_FOLDER, filename)

try: 
    output_data = read_file(OUTPUT_FOLDER, filename)
except:
    output_data = None

# Load model building blocks
weights, list_prev_size, list_layer_size, list_activation = model_load(data)

# Build NN Model
act = {0: linear, 1: ReLU, 2: sigmoid, 3: softmax}
net = NeuralNetwork()
for i in range (int(data[0]) - 1):
    net.add(ConnectedLayer(list_prev_size[i], list_layer_size[i], weights[i]))
    net.add(ActivationLayer(act[list_activation[i]]))

# Generate x_train
x_train = generate_x_train(input_data)
out, complete_out = predict_output(x_train, net)

# Print the information, complete with brief verbose
print('\n---Prediction Information---')
print('Input data :', x_train)

# Compute out error
out_pred, out_true, sse_err, isLessThanMaxSSE = compute_out_and_errors(output_data, out)

print('Output prediction :', out_pred)
print('Output true :', out_true)
print('SSE  : ', sse_err)
print("sse <= max_sse  :", isLessThanMaxSSE)


x_train = x_train
y_train = out_true.reshape(len(x_train), list_layer_size[-1])

---Model Information---
Number of layers : 3
Input size : 2
Output size : 2

Weights : [[[0.1, 0.2], [0.4, -0.5], [0.7, 0.8]], [[0.1, 0.2], [0.4, -0.5], [0.7, 0.8]]]
Previous layer size : [2, 2]
Current layer size : [2, 2]
Activation function : [0, 1]

---Prediction Information---
Input data : [[ 1.  -1.   0.5]
 [ 1.   0.5 -1. ]]
Output prediction : [0.89  1.055 0.    0.   ]
Output true : [0.1 1.  1.  0.1]
SSE  :  0.8185625000000001
sse <= max_sse  : False


Run if process input from iris

In [171]:
# load iris import datasets first
from sklearn import datasets
iris = datasets.load_iris()

x_train = np.insert(iris.data, 0, 1, axis=1)[:5]
y_train = iris.target.reshape(-1,1)[:5]

Run if process input from .txt, not iris dataset

In [228]:
x_train = x_train
y_train = out_true.reshape(len(x_train), list_layer_size[-1])
print("x_train",x_train)
print("y_train",y_train)

x_train [[ 1.  -1.   0.5]
 [ 1.   0.5 -1. ]]
y_train [[0.1 1. ]
 [1.  0.1]]


In [233]:
act = {0: linear, 1: ReLU, 2: sigmoid, 3: softmax}
net = NeuralNetwork()
for i in range (int(data[0]) - 1):
    net.add(ConnectedLayer(list_prev_size[i], list_layer_size[i], weights[i]))
    net.add(ActivationLayer(act[list_activation[i]]))

net.fit(x_train, y_train, learning_rate=0.1, epochs=1, batch_size=2, err_threshold=0.0)

WEIGHT ORI
|____Layer 0___|
---B I A S---
[0.1, 0.2]
---N E U R O N S---
[[0.4, -0.5], [0.7, 0.8]]


|____Layer 1___|
---B I A S---
[0.1, 0.2]
---N E U R O N S---
[[0.4, -0.5], [0.7, 0.8]]


initial error= 0.8185625000000001

Activation Layers From Back to Front:  ['ReLU', 'linear']

============BATCH 1==============
====FORWARD PROPAGATION====
Batch X Train
[[ 1.  -1.   0.5]
 [ 1.   0.5 -1. ]]
Batch Y Train
[[0.1 1. ]
 [1.  0.1]]
Batch Y pred
[[0.89  1.055]
 [0.    0.   ]]

===Batch 1 Sample 1===

--Output Materials--
sample_y_pred
[0.89  1.055]
sample_y_train
[0.1 1. ]
sample_pred_out
[[1.   1.  ]
 [0.05 0.05]
 [1.1  1.1 ]]
Output Layer Type
ReLU

--Calculation ReLU---
dE_dO
[0.79  0.055]
dO_dNet
[1 1]
dNet_dW
[[1.   1.  ]
 [0.05 0.05]
 [1.1  1.1 ]]
dE_dW
[[0.79    0.055  ]
 [0.0395  0.00275]
 [0.869   0.0605 ]]

===Batch 1 Sample 2===

--Output Materials--
sample_y_pred
[0. 0.]
sample_y_train
[1.  0.1]
sample_pred_out
[[ 1.    1.  ]
 [-0.4  -0.4 ]
 [-0.85 -0.85]]
Output Layer Type
R